In [1]:
# This is the training script for fine-tuning bert on 
# unaltered GPT data and manually labelled data
# For better performance/generalization, look for augmented dataset
# Read README.md for comments and details.

In [2]:

# all classes

classes = ["banking","valuation","household","real estate","corporate","external","sovereign","technology", "climate", "energy", "health", "eu"]


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,top_k_accuracy_score
import math
import pickle
from datasets import Dataset

In [4]:
# load bert-based and finbert
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(classes))
finbert = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert', use_fast =True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
# weights transfer for encoder layers only 
finbert_weights = finbert.state_dict()
model_weights = model.state_dict()
del finbert_weights["bert.pooler.dense.weight"]
del finbert_weights["bert.pooler.dense.bias"]
del finbert_weights["classifier.weight"]
del finbert_weights["classifier.bias"]
finbert_weights["bert.pooler.dense.weight"] = model_weights["bert.pooler.dense.weight"]
finbert_weights["bert.pooler.dense.bias"] = model_weights["bert.pooler.dense.bias"]
finbert_weights["classifier.weight"] = model_weights["classifier.weight"]
finbert_weights["classifier.bias"] = model_weights["classifier.bias"]

model.load_state_dict(finbert_weights)


<All keys matched successfully>

In [6]:
# flatten to one list for all 3

# manual labelled
with open('train_data.pickle', 'rb') as file:
    train = pickle.load(file)

# gpt labelled p1
with open('gpt.pickle', 'rb') as file:
    gpt = pickle.load(file)

# gpt labelled p2
with open('gpt_p2.pickle', 'rb') as file:
    gpt2 = pickle.load(file)
gpt = [item for sublist in gpt for item in sublist]
gpt2 = [item for sublist in gpt2 for item in sublist]

mixed = gpt + gpt2

In [7]:
print(len(mixed))

2458


In [8]:
sample = 1

additional_text = []
additional_label = []

for idx in range(1,len(mixed)-1):
    sent1, sent2 = mixed[idx-1:idx+1]
    for i in range(1):
        additional_text.append(sent1["text"]+' '+ sent2["text"])
        dist = (np.array(sent1["dist"]) + np.array(sent2["dist"]))/2
        additional_label.append(np.argmax(dist))

for idx in range(2,len(mixed)-1):
    sent1, sent2, sent3 = mixed[idx-2:idx+1]
    for i in range(1):
        additional_text.append(sent1["text"]+' '+ sent2["text"]+' '+sent3["text"])
        dist = (np.array(sent1["dist"]) + np.array(sent2["dist"])+ np.array(sent3["dist"]))/3
        additional_label.append(np.argmax(dist))

text_max = [item["text"] for i in range(sample) for item in train]
label_max = [np.argmax(item["dist"]) for i in range(sample) for item in train]

text_max_mixed = [item["text"] for i in range(sample) for item in mixed]
label_max_mixed = [np.argmax(item["dist"]) for i in range(sample) for item in mixed]


In [9]:
import random
def randomize(text, label):
    temp = list(zip(text, label))
    random.shuffle(temp)
    comb_text, comb_label =  zip(*temp)
    return comb_text, comb_label

a1 = randomize(text_max, label_max)
a2 = randomize(text_max_mixed, label_max_mixed)
a3 = randomize(additional_text, additional_label)

test_text, train_text =  [*a1[0][math.ceil(len(a1[0])*0.8):],*a2[0][math.ceil(len(a2[0])*0.8):],*a3[0][math.ceil(len(a3[0])*0.8):] ], [*a1[0][:math.ceil(len(a1[0])*0.8)],*a2[0][:math.ceil(len(a2[0])*0.8)],*a3[0][:math.ceil(len(a3[0])*0.8)] ]
test_label, train_label = [*a1[1][math.ceil(len(a1[1])*0.8):],*a2[1][math.ceil(len(a2[1])*0.8):],*a3[1][math.ceil(len(a3[1])*0.8):] ],  [*a1[1][:math.ceil(len(a1[1])*0.8)],*a2[1][:math.ceil(len(a2[1])*0.8)],*a3[1][:math.ceil(len(a3[1])*0.8)] ]


In [10]:
print(len(train_text))
print(len(test_text))

6105
1525


In [11]:
assert len(train_text) == len(train_label)
assert len(test_text) == len(test_label)

In [12]:
train_dataset = Dataset.from_dict({"text":train_text, "label":train_label})
test_dataset = Dataset.from_dict({"text":test_text, "label":test_label})

In [13]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

100%|██████████| 1/1 [00:00<00:00,  8.85ba/s]


In [14]:
train_dataset["input_ids"]

tensor([[  101,  2152,  7016,  ...,     0,     0,     0],
        [  101,  1996,  2204,  ...,     0,     0,     0],
        [  101,  2000,  9585,  ...,     0,     0,     0],
        ...,
        [  101,  2339,  2106,  ...,     0,     0,     0],
        [  101,  1998,  2023,  ...,     0,     0,     0],
        [  101,  2005, 12194,  ...,     0,     0,     0]])

In [15]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    top3 = top_k_accuracy_score(labels, pred.predictions,k=3)
    top2 = top_k_accuracy_score(labels, pred.predictions,k=2)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'top3': top3,
        'top2': top2
    }

training_args = TrainingArguments(
    
    output_dir='./results',
    learning_rate=2e-5,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    metric_for_best_model="accuracy",
    evaluation_strategy='epoch',
    save_strategy = "epoch",
    logging_dir='./logs',
    save_total_limit = 1, # Only last 5 models are saved. Older ones are deleted.
    load_best_model_at_end=True,
)
   
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [16]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 6105
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7640
  5%|▌         | 382/7640 [01:00<18:29,  6.54it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1525
  Batch size = 64
C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

{'eval_loss': 1.3231501579284668, 'eval_accuracy': 0.6216393442622951, 'eval_f1': 0.2641283468775932, 'eval_precision': 0.24853064605410116, 'eval_recall': 0.2845097464096975, 'eval_top3': 0.838688524590164, 'eval_top2': 0.7704918032786885, 'eval_runtime': 3.7587, 'eval_samples_per_second': 405.724, 'eval_steps_per_second': 6.385, 'epoch': 1.0}


Model weights saved in ./results\checkpoint-382\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-1] due to args.save_total_limit
  7%|▋         | 501/7640 [01:27<17:59,  6.61it/s]

{'loss': 1.8285, 'learning_rate': 2e-05, 'epoch': 1.31}


 10%|█         | 764/7640 [02:09<15:31,  7.38it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1525
  Batch size = 64
C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 10%|█         | 764/7640 [02:12<15:31,  7.38it/s]Saving model checkpoint to ./results\checkpoint-764
Configuration saved in ./results\checkpoint-764\config.json


...

{'loss': 0.0237, 'learning_rate': 1.0196078431372549e-05, 'epoch': 10.47}


 55%|█████▌    | 4202/7640 [12:37<07:46,  7.37it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1525
  Batch size = 64

 55%|█████▌    | 4202/7640 [12:40<07:46,  7.37it/s]Saving model checkpoint to ./results\checkpoint-4202
Configuration saved in ./results\checkpoint-4202\config.json


{'eval_loss': 1.30657160282135, 'eval_accuracy': 0.8045901639344263, 'eval_f1': 0.7008775117123527, 'eval_precision': 0.7653860892522012, 'eval_recall': 0.6744593488621988, 'eval_top3': 0.9311475409836065, 'eval_top2': 0.9009836065573771, 'eval_runtime': 3.2293, 'eval_samples_per_second': 472.238, 'eval_steps_per_second': 7.432, 'epoch': 11.0}


Model weights saved in ./results\checkpoint-4202\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-3820] due to args.save_total_limit
 57%|█████▋    | 4330/7640 [13:03<11:05,  4.97it/s]

RuntimeError: transform: failed to synchronize: cudaErrorLaunchFailure: unspecified launch failure

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1526
  Batch size = 64
100%|██████████| 24/24 [00:03<00:00,  6.37it/s]


{'eval_loss': 1.3667666912078857,
 'eval_accuracy': 0.8289646133682831,
 'eval_f1': 0.7600767747856746,
 'eval_precision': 0.7783009182422279,
 'eval_recall': 0.74921395397379,
 'eval_top3': 0.936435124508519,
 'eval_top2': 0.9121887287024901,
 'eval_runtime': 3.9514,
 'eval_samples_per_second': 386.188,
 'eval_steps_per_second': 6.074,
 'epoch': 20.0}

In [ ]:

predict_dataset = Dataset.from_dict({"text":["In contrast to the radical forces buffeting valuations, for most companies, 2020 was a year of “strategy lockdown.",
"Domestic policies thus tended to reinforce negative spillovers and exacerbate systemic risk across the euro area.",
"Mortgage interest rate in selected European countries as of 4th quarter of 2019 and 2020 increased",
"Accordingly, I shall spend most of my allotted time outlining the chosen monetary policy instruments and procedures of the ESCB and the considerations that have been raised relating to strategy.",
"Credit card interest rate has gone up a lot."]})

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
predict_dataset = predict_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))

trainer.predict(predict_dataset)

100%|██████████| 1/1 [00:00<00:00, 167.14ba/s]
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 5
  Batch size = 64
  0%|          | 0/1 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-0.54258686, -2.283698  , -1.2337625 ,  0.6101017 ,  8.834227  ,
        -0.60477626, -0.1470656 , -0.25806686, -1.9158627 , -0.45982993,
        -0.08515993, -1.3472333 ],
       [ 9.702999  , -0.29276183, -1.5581697 , -1.5263258 , -1.2451688 ,
        -0.5294447 ,  2.195682  , -1.8384721 , -1.5489938 , -3.0295794 ,
        -1.7577528 , -1.730811  ],
       [ 1.4455788 , -1.5327429 ,  2.143541  ,  7.9250283 , -1.1390123 ,
        -3.0250149 , -0.9481666 , -2.0208018 ,  0.02337106, -0.8389467 ,
        -0.08845273, -0.9033677 ],
       [-1.1007565 , -1.3134553 , -1.9952139 , -0.77075744, -1.4220358 ,
         0.21309426, 10.992527  , -0.8264481 , -0.6472416 , -2.5305545 ,
        -1.0293103 , -1.4311087 ],
       [ 2.7709064 ,  7.787258  ,  2.5337188 , -1.0464295 , -2.2166548 ,
        -1.2778659 , -2.6960404 , -2.199739  , -1.8598685 , -1.4836167 ,
        -1.3202602 , -1.9072998 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 0.0389, 